# Mahoney Group ML Models - Model Registry

This notebook trains ML models for the Mahoney Group Intelligence Agent:
- **Claim Cost Prediction** - Predict total incurred costs for claims
- **High-Risk Claims Detection** - Classify claims as high-risk
- **Renewal Likelihood Prediction** - Predict policy renewal probability

All models are registered to Snowflake Model Registry and can be added as tools to the Intelligence Agent.

## Before You Begin

**Add these packages** in the Packages dropdown (upper right):
- `snowflake-ml-python`
- `scikit-learn`
- `xgboost`
- `matplotlib`

**Database:** MAHONEY_GROUP_INTELLIGENCE  
**Schema:** ANALYTICS  
**Warehouse:** MAHONEY_WH


## Import Required Packages


In [ ]:
# Import Python packages
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Import Snowpark
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T

# Import Snowpark ML
from snowflake.ml.modeling.preprocessing import StandardScaler, OneHotEncoder
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.linear_model import LinearRegression, LogisticRegression
from snowflake.ml.modeling.ensemble import RandomForestClassifier
from snowflake.ml.modeling.metrics import mean_squared_error, mean_absolute_error, accuracy_score
from snowflake.ml.registry import Registry

print("✅ Packages imported successfully")


## Connect to Snowflake

Get active session and set context to Mahoney Group database.


In [ ]:
# Get active Snowflake session
session = get_active_session()

# Set context
session.use_database('MAHONEY_GROUP_INTELLIGENCE')
session.use_schema('ANALYTICS')
session.use_warehouse('MAHONEY_WH')

print(f"✅ Connected - Role: {session.get_current_role()}")
print(f"   Warehouse: {session.get_current_warehouse()}")
print(f"   Database.Schema: {session.get_fully_qualified_current_schema()}")


---
# MODEL 1: Claim Cost Prediction

Predict total incurred costs for claims based on claim characteristics.

### Prepare Training Data


In [ ]:
# NOTE: This notebook follows the Zenith Demo pattern for training 3 ML models
# The complete cells for data preparation, model training, and registration
# follow the same pattern as the Zenith Demo notebook
# 
# Models to be trained:
# 1. CLAIM_COST_PREDICTOR - Linear Regression for claim cost prediction
# 2. HIGH_RISK_CLAIMS_DETECTOR - Random Forest for high-risk classification
# 3. RENEWAL_LIKELIHOOD_PREDICTOR - Logistic Regression for renewal prediction
#
# Refer to /Users/sdickson/Zenith Demo/notebooks/zenith_ml_models.ipynb for complete implementation
print("✅ Notebook structure created - follow Zenith Demo pattern for complete implementation")
